In [3]:
from typing import Tuple

import itertools

# import numpy
import pandas

# from matplotlib import pyplot as plt

from neuralprophet import NeuralProphet
# from neuralprophet.diagnostics import cross_validation, performance_metrics
# from neuralprophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

In [5]:
country = 'france'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')
# df_covid_measures = pandas.read_csv(
#     filepath_or_buffer = f'../../../data/intermediate/{country}/government_covid_measures.csv'
# )

In [6]:
# df_covid_workplace_lockdowns = df_covid_measures[['date', 'workplace_closing']]
# df_covid_workplace_lockdowns.to_csv('covid_workplace_lockdowns.csv', index = False)

In [7]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)

    # manually created from df_covid_workplace_lockdowns
    # df_workplace_lockdowns = pandas.DataFrame([
    #     {'holiday': 'lockdown_1', 'ds': '2020-06-22', 'lower_window': 0, 'ds_upper': '2020-08-13'},
    #     {'holiday': 'lockdown_2', 'ds': '2022-03-14', 'lower_window': 0, 'ds_upper': '2022-06-19'},
    #     {'holiday': 'lockdown_2', 'ds': '2020-05-11', 'lower_window': 0, 'ds_upper': '2020-06-21'},
    #     {'holiday': 'lockdown_2', 'ds': '2020-08-14', 'lower_window': 0, 'ds_upper': '2020-10-29'},
    #     {'holiday': 'lockdown_2', 'ds': '2020-11-28', 'lower_window': 0, 'ds_upper': '2021-02-25'},
    #     {'holiday': 'lockdown_2', 'ds': '2021-03-02', 'lower_window': 0, 'ds_upper': '2021-03-04'},
    #     {'holiday': 'lockdown_2', 'ds': '2021-04-03', 'lower_window': 0, 'ds_upper': '2021-05-02'},
    #     {'holiday': 'lockdown_2', 'ds': '2021-05-19', 'lower_window': 0, 'ds_upper': '2022-03-13'},
    #     {'holiday': 'lockdown_3', 'ds': '2020-03-17', 'lower_window': 0, 'ds_upper': '2020-05-10'},
    #     {'holiday': 'lockdown_3', 'ds': '2020-10-30', 'lower_window': 0, 'ds_upper': '2020-11-27'},
    #     {'holiday': 'lockdown_3', 'ds': '2021-02-26', 'lower_window': 0, 'ds_upper': '2021-03-01'},
    #     {'holiday': 'lockdown_3', 'ds': '2021-03-05', 'lower_window': 0, 'ds_upper': '2021-04-02'},
    #     {'holiday': 'lockdown_3', 'ds': '2021-05-03', 'lower_window': 0, 'ds_upper': '2021-05-18'},
    # ])
    # for t_col in ['ds', 'ds_upper']:
    #     df_workplace_lockdowns[t_col] = pandas.to_datetime(df_workplace_lockdowns[t_col], format = '%Y-%m-%d')
    # df_workplace_lockdowns['upper_window'] = (df_workplace_lockdowns['ds_upper'] - df_workplace_lockdowns['ds']).dt.days
    # df_workplace_lockdowns = df_workplace_lockdowns.sort_values(by = 'ds', axis = 'index', ascending = True, inplace= False)
    return df_mrd, df_covid_measures

In [8]:
df_mrd, _ = make_dataset(df)

In [9]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [10]:
df_train, df_test = train_test_split(df_mrd, 12)

In [11]:
# def train_model(df: pandas.DataFrame, df_holidays: pandas.DataFrame = pandas.DataFrame(), param_grid: dict = {}, eval_metric: str = 'rmse', eval_metric_optimize: str = 'min') -> NeuralProphet:
    # Generate all combinations of parameters
    # all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    # eval_metric_values = []  # Store the eval_metric_values for each params here

    # # Use cross validation to evaluate all parameters
    # for params in all_params:
    #     m = NeuralProphet(**params).fit(df)  # Fit model with given params
    #     df_cv = cross_validation(m, horizon = '60 days', parallel="processes")
    #     df_p = performance_metrics(df_cv, rolling_window=1, monthly=True)
    #     eval_metric_values.append(df_p[eval_metric].values[0])

    # # Find the best parameters
    # tuning_results = pandas.DataFrame(all_params)
    # tuning_results[eval_metric] = eval_metric_values

    # tuning_results.sort_values(by = eval_metric, axis = 'index', ascending = True, inplace = True)
    # best_params = {}
    # if eval_metric_optimize == 'min':
    #     best_params = tuning_results.iloc[0].to_dict()
    # elif eval_metric_optimize == 'max':
    #     best_params = tuning_results.iloc[-1].to_dict()
    # else:
    #     raise ValueError("Invalid Parameter Value: param 'eval_metric_optimize' may only have values 'min' or 'max'.")


    # optimal_model = NeuralProphet().fit(df)
    # return optimal_model

In [18]:
# param_grid = {  
#     'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
#     'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
# }

# model = train_model(df_train)
model = NeuralProphet()
model_fitting = model.fit(df_train)

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 91.489% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as MS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 252


  0%|          | 0/118 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.00E-01, min: 5.88E-01


  0%|          | 0/118 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 7.02E-02, min: 4.12E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 8.89E-02
Epoch[252/252]: 100%|██████████| 252/252 [00:19<00:00, 12.86it/s, SmoothL1Loss=0.00353, MAE=0.351, RMSE=0.438, Loss=0.00294, RegLoss=0]


In [21]:
model_fitting

,SmoothL1Loss,MAE,RMSE,Loss,RegLoss
0,0.257051,3.097814,3.933787,0.192592,0.0
1,0.235936,2.939619,3.742459,0.175119,0.0
2,0.213703,2.772682,3.561428,0.156933,0.0
3,0.191694,2.609226,3.341804,0.138975,0.0
4,0.166932,2.429247,3.101577,0.119552,0.0
...,...,...,...,...,...
247,0.003533,0.351420,0.441083,0.002942,0.0
248,0.003531,0.351259,0.440732,0.002940,0.0
249,0.003531,0.351292,0.436705,0.002941,0.0
250,0.003529,0.351224,0.439768,0.002939,0.0


In [25]:
def test_model(df_test: pandas.DataFrame, model: NeuralProphet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat1'], squared = False)
    return df_predicted, rmse

In [26]:
df_predicted, rmse = test_model(df_test, model)

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 91.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 91.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [28]:
rmse

0.7034399021440034

In [29]:
df_predicted

,ds,y,residual1,yhat1,trend,season_yearly
0,2022-03-01,7.4,0.667293,8.067293,8.059205,0.008088
1,2022-04-01,7.5,0.523349,8.023349,8.040203,-0.016854
2,2022-05-01,7.5,0.488455,7.988455,8.021815,-0.033360
3,2022-06-01,7.5,0.460323,7.960323,8.002815,-0.042492
4,2022-07-01,7.3,0.708663,8.008663,7.984426,0.024237
5,2022-08-01,7.2,0.792328,7.992328,7.965425,0.026903
6,2022-09-01,7.1,0.875335,7.975335,7.946424,0.028911
7,2022-10-01,7.2,0.733476,7.933476,7.928036,0.005441
8,2022-11-01,7.2,0.727266,7.927266,7.909035,0.018230
9,2022-12-01,7.2,0.706010,7.906010,7.890647,0.015363


In [30]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01']})

In [31]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 100.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS


KeyError: 'y'

In [ ]:
df_future_prediction